In [62]:

%%matplotlibmatplotli  inline
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from scipy import stats
from ast import literal_eval # 문자열 모형의 딕트를 스근하게 딕트로 바꾸어 준다. 
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from surprise import Reader, Dataset, SVD, evaluate

UsageError: Cell magic `%%matplotlibmatplotli` not found.


In [63]:
dataSet = pd.read_csv('crawler/top.csv')
dataSet = dataSet.drop(columns='Unnamed: 0', axis=1)
dataSet2 = dataSet.copy()
reviewData = dataSet[['cosmeticId', 'type', 'review']]

In [64]:
reviewArray = ['' for i in range(100)]
for index, row in reviewData.iterrows():
    reviewArray[row['cosmeticId']] += row['review']

In [65]:
name_popId = dataSet[['name', 'cosmeticId']]
id2name = dataSet[['cosmeticId', 'name']]
name_popId = name_popId.drop_duplicates().set_index('name')
id2name = id2name.drop_duplicates().set_index('cosmeticId')

In [66]:
dataSet = dataSet.drop('name', axis=1)

In [67]:
dataSet.head(5)
seriesReview = pd.Series(reviewArray)

In [68]:
tf = TfidfVectorizer(analyzer='word', min_df=2, stop_words=['\r', '\n'], sublinear_tf=True)
tf_matrix = tf.fit_transform(seriesReview)

In [69]:
tf_matrix

<100x4970 sparse matrix of type '<class 'numpy.float64'>'
	with 31625 stored elements in Compressed Sparse Row format>

In [70]:
cosine_sim = linear_kernel(tf_matrix, tf_matrix)
cosine_sim[0]
reader = Reader()

In [71]:
dataSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1923 entries, 0 to 1922
Data columns (total 4 columns):
cosmeticId    1923 non-null int64
rate          1923 non-null float64
review        1923 non-null object
type          1923 non-null object
dtypes: float64(1), int64(1), object(2)
memory usage: 60.2+ KB


In [72]:
type = {'건성': 0, '지성': 1, '중성': 2, '복합성': 3, '민감성': 4}
dataSet['type'] = dataSet['type'].map(type)
trainData = dataSet[['type', 'cosmeticId', 'rate']].drop_duplicates()
data = Dataset.load_from_df(trainData , reader)

In [73]:
data.split(n_folds=5)

In [74]:
svd = SVD()
evaluate(svd, data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.3592
MAE:  0.2390
------------
Fold 2
RMSE: 0.3168
MAE:  0.2307
------------
Fold 3


RMSE: 0.2929
MAE:  0.2146
------------
Fold 4
RMSE: 0.3107
MAE:  0.2252
------------
Fold 5
RMSE: 0.2938
MAE:  0.2320
------------
------------
Mean RMSE: 0.3147
Mean MAE : 0.2283
------------
------------


C:\Users\LeeSangHoon\Anaconda3\envs\tensorflow\lib\site-packages\surprise\evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
C:\Users\LeeSangHoon\Anaconda3\envs\tensorflow\lib\site-packages\surprise\dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.3592496132284334,
                             0.3167578957123547,
                             0.29285559883540546,
                             0.31065968845085445,
                             0.29377102561753393],
                            'mae': [0.23903822900520943,
                             0.23074744670072084,
                             0.21459471869623092,
                             0.2252203421215638,
                             0.23195913234777632]})

In [75]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [76]:
def type_cosmeticsNum(type, cosmeticName=''):
    if cosmeticName != '':
        idx = name_popId['cosmeticId'][cosmeticName]
        print(idx)
    else:
        sortRate = dataSet[dataSet['type'] == type].drop_duplicates(['cosmeticId'])
        sortRate = sortRate.sort_values('rate', ascending=False)
        idx = sortRate['cosmeticId'][:10]
        sortRate = sortRate.reset_index(drop=True)
        sortRate['lenReview'] = 0
        for i, k in enumerate(idx):
            sortRate.loc[i, 'lenReview'] = len(dataSet2[dataSet2['cosmeticId'] == k]['review'])
        sortRate = sortRate.sort_values('lenReview', ascending=False)
        idx = sortRate['cosmeticId'][:1]
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    cosmetic_indcies = [i[0] for i in sim_scores]
    cosmeticpd = pd.Series(cosmetic_indcies)

    # #cosmetics = dataSet.iloc[cosmetic_indcies][['cosmeticId', 'rate', 'review', 'type']]
    cosmeticpd = {'cosmeticId': cosmetic_indcies, 'est': cosmeticpd.apply(lambda x: svd.predict(type, x).est
                                                                          )}
    cosmeticpd = pd.DataFrame(cosmeticpd, columns=['cosmeticId', 'est'])
    cosmeticpd = cosmeticpd.sort_values('est', ascending=False)

    # finalCosmetics = cosmetics.drop_duplicates(['cosmeticId'])
    # finalCosmetics = finalCosmetics['cosmeticId'].reset_index(drop=True)
    # preCosmeticName = name_popId.index[finalCosmetics]
    for id in cosmeticpd['cosmeticId']:
        print(id2name.loc[id]['name'])

In [96]:
type_cosmeticsNum(0, '')

내추럴 미네랄 썬 프로텍션
퓨어 미네랄 선크림 SPF50+ PA++++
산소수 라이트 선 젤 SPF50+ PA++++
마일드 프로텍트 순한 선크림 SPF50+ PA+++
UV 아쿠아 리치 워터리 에센스 선크림 SPF50+ PA++++
마일드 프로텍트 에센스 선 SPF50+ PA++++
스킨 디펜스 멀티 프로텍션 에센스 SPF50+ PA+++ 
산소수 피팅 선 크림 SPF50+ PA+++
NEW 이데베논 선블럭 SPF50+PA++++
마일드-S 모이스쳐 라이트 선 젤 SPF50+ PA++++
UV 퍼펙트 에어쿨 선 젤 SPF50+ PA++++
유브이 디펜스 미 블루 레이 선 젤 SPF50+ PA++++
에코 어스 파워 노 세범 선 크림 SPF50+ PA+++
유브이 스킨포스 선크림 SPF50+ PA+++
아임 쏘리 포 마이 스킨 데일리 카밍 선 크림 SPF50+ PA++++
모이스쳐라이징 썬 스크린 SPF50+ PA+++
퓨어 블록 내추럴 데일리 선크림 SPF45 PA+++
유브이 디펜스 미 블루 레이 선 플루이드 SPF50+ PA++++
블랙 티 올 데이 아쿠아 선 젤 SPF30 PA++
365 선 크림 SPF45 PA+++
피토 아쿠아 워터 선 베이스 SPF50+ PA+++
선 솔루션 프레쉬 선 에센스 IRF20 SPF50+ PA++++
더마 디펜스 선 플루이드
워터 스플래쉬 선크림 SPF50+ PA+++
선크림 스테이 선 세이프 SPF50+ PA++++


In [80]:
reviewArray

['일본 비오레 아쿠아 썬크림 다 써서 최대한 비슷한거 찾아봤더니 이 제품이 제일 비슷해서 구매했습니다.\r\n백탁없고, 끈적임없고, 수분로션같은 제형 딱 제가 원하는 선크림이라서 만족해요!\r\n이런 선크림이 화장할때 걸리적 거리지도 않고 좋아요.\r\n그리고 수분 로션 같은 제형이 더 화장이 잘먹는거 같고 밀리지도 않고 넘 괜찮네요! 비오레 아쿠아 선크림만 3통째 썼었는데 이젠 갈아타도될듯!\r\n다른 사람들 리뷰보니 눈시림이 있다던데 저는 잘 모르겠더라구요? 뭐 나중에 눈시림 있을까봐 그건 걱정되네요ㅠㅠ글픽보고 세일때 샀는데 끈적임없고 흡수가 빨랐어요!! 향도 레몬?비타민?같은 향이라서 좋았고 원래 건성 이라고 보습있는것만 썼더니 너무 끈적여서 싫었는데 이건 건조하지두 않고 걍 머 바른느낌 별로안나서 좋은거같아요~우오 나 이거 3위일때인가 샀는데 지금 1위네영?\r\n가지고 다니던 썬크림을 잃어버려서 급하게 지르느라 글로우픽 참고해서 지른 제품.\r\n역시 믿고보는 글로우픽! 딱 제가 원하는 제형이에요. 발림성 좋고 가볍고 끈적이지 않고!(제일중요!!)\r\n화장할때 보통 선크림 기름기 땜에 잘 뜨는데 이건 그렇지도 않았어요~ 백탁현상도 거의 없었던거 같음.\r\n또 하나! 입구가 삐쭉해서 좋아요 양조절이 확실히 되는듯!일단 제형이 수분감 있어서 좋구 백탁없구 쓰기 너무 좋아요 하지만..저에겐 눈시림이 살짝 아주 살짝 있어서 좀 아쉬워용ㅠ끈적임없고 세미매트한 마무리감 톤업이 되지않는 제품이라서 팔에 바르기가 좋고 메이크업전에 선케어 베이스로 바르기에도 화장이 밀리지않고 베리굿!! 향도 나름 괜찮아서 잘쓰고 있어요남자친구도 잘 쓰고 있는 선크림이에요! 자극성도 없구! 예민한 피부에 딱이에요 ??식물나라 선젤이랑 비슷하다는 후기보고\r\n바로 올영가서 테스트해봄\r\n\r\n식물나라 선젤을 너무 잘써서\r\n재구매하려던 참에\r\n올영에 나란히 놓여있길래\r\n손등에 비교해보니\r\n차이를 잘 모를정도로 똑같달까\r\n\r\n가격은 얘가 아주 조금 더 저렴했

In [78]:
dataSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1923 entries, 0 to 1922
Data columns (total 4 columns):
cosmeticId    1923 non-null int64
rate          1923 non-null float64
review        1923 non-null object
type          1923 non-null int64
dtypes: float64(1), int64(2), object(1)
memory usage: 60.2+ KB
